# **Educational Chatbot: Utilizing LLama 2 chatbot technology to deliver responses to educational questions, and recommending relevant articles leveraging LLM - based keyword analysis of chatbot responses.**

## Installing accelerate and authenticating hugging face login, to use LLama 2 7B chatbot model


In [ ]:
!pip install transformers torch accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 4.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!huggingface-cli whoami

sanjanaprasadkr


## Loading the model and tokenizer and creating the LLama 2 7B pipeline

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf" # meta-llama/Llama-2-7b-hf

tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:757: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline

llama_pipeline = pipeline(
    "text-generation",  # LLM task
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

#By using 16-bit floating point format (float16), the memory usage during inference can be reduced compared to the default 32-bit floating point format
# pipeline should automatically select the appropriate device (CPU or GPU) for running inference based on availability

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

## Getting the chatbot response to a question asked by the user

In [ ]:
def get_llama_response(prompt: str) -> str:
    """
    Generate a response from the Llama model.

    Parameters:
        prompt (str): The user's input/question for the model.

    Returns:
        str: The model's response.
    """
# Generate sequences using the Llama pipeline
    sequences = llama_pipeline(
        prompt,  # User's input/question
        do_sample=True,  # Use sampling-based decoding
        top_k=10,  # Consider top-10 tokens during sampling
        num_return_sequences=1,  # Generate one sequence
        eos_token_id=tokenizer.eos_token_id,  # End-of-sequence token ID
        max_length=1000,  # Maximum length of the generated sequence
    )
    # Return the generated response
    return sequences[0]['generated_text']

prompt = 'Can you explain what dark matter is?\n'
doc2 = get_llama_response(prompt)
print("Chatbot:", doc2)


Chatbot: Can you explain what dark matter is?

Dark matter is a hypothetical form of matter that is thought to exist in the universe but cannot be directly detected by telescopes or other astronomical instruments. It is called "dark" because it does not emit, absorb, or reflect any electromagnetic radiation, such as light, making it invisible to our telescopes. Despite its elusive nature, dark matter's presence can be inferred through its gravitational effects on visible matter and the way galaxies and galaxy clusters move.

The existence of dark matter was first proposed by Swiss astrophysicist Fritz Zwicky in the 1930s, and since then, a wealth of observational evidence has accumulated to support its existence. Some of the key lines of evidence include:

1. Galactic rotation curves: The rotation curves of galaxies are the rate at which stars and gas orbit around the center of the galaxy. These curves are typically flat or even rise slightly towards the edges of the galaxy. This is un

In [ ]:
%config Completer.use_jedi=False

## Installing KeyBERT for keyword detection of chatbot response - this will help in scraping and displaying relevant articles

In [ ]:
!pip install KeyBERT

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.5 MB/s eta 0:00:00
  Created wheel for KeyBERT: filename=keybert-0.8.4-py3-none-any.whl size=39200 sha256=b0ecdb2f96721b57fd97ed7a0301fca600e7492bd4e31b07c008d1d071ba2086
  Stored in directory: /root/.cache/pip/wheels/97/ef/4c/6588bd7072b0cc04225b40e639b991e49ebd4e21fb81f0acee
Successfully built KeyBERT


In [ ]:
from keybert import KeyBERT

In [ ]:
model = KeyBERT(model="distilbert-base-nli-mean-tokens") #distilled version of the BERT model, which is smaller and faster but still maintains good performance

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/550 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Important keywords, and their relevance score

In [ ]:
%%time
model.extract_keywords(
    doc2,
    top_n=10, #considers the top 10 keywords
    keyphrase_ngram_range=(1, 1), #1 word
    stop_words="english", #removes stopwords such as "the"
)

CPU times: user 116 ms, sys: 15.6 ms, total: 132 ms
Wall time: 280 ms


[('astrophysicist', 0.4303),
 ('neutrinos', 0.384),
 ('telescopes', 0.3555),
 ('galaxies', 0.3535),
 ('cosmic', 0.3365),
 ('radiation', 0.3264),
 ('microwave', 0.3161),
 ('black', 0.2946),
 ('astronomical', 0.2933),
 ('invisible', 0.2776)]

## Using BeautifulSoup for web scraping of relevant articles, and displaying their links

In [ ]:
import requests
from bs4 import BeautifulSoup


# Extract keywords using KeyBERT
keywords = model.extract_keywords(doc2, top_n=10, keyphrase_ngram_range=(1, 1), stop_words="english")
keywords = [keyword[0] for keyword in keywords]

# Define target website
base_url = "https://www.livescience.com/"

# Function to extract articles and their URLs
def extract_articles_with_urls(soup, keywords):
    articles_with_urls = []
    # Find elements containing article headings
    for heading in soup.find_all("h3", class_="article-name"):
        title = heading.text.strip()
        # Check if any keyword is present in the title (case-insensitive)
        if any(keyword.lower() in title.lower() for keyword in keywords):
            # Extract URL from parent <a> tag
            article_url = heading.find_parent("a")["href"]
            articles_with_urls.append((title, article_url))
    return articles_with_urls

# Get the main page content
response = requests.get(base_url)

# Check for successful response
if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    articles_with_urls = extract_articles_with_urls(soup, keywords)
    if articles_with_urls:
        print("Recommended articles:")
        for title, url in articles_with_urls:
            # Check if the URL starts with the base_url
            if url.startswith(base_url):
                # Remove the base_url from the URL
                url = url[len(base_url):]  # Remove the base_url
                # Add back the base_url to the modified URL
                url = base_url + url
            print(f"{title}: {url}")
    else:
        print("No articles found matching the extracted keywords.")
else:
    print(f"Error: Failed to access {base_url}")

Recommended articles:
Scientists are one step closer to knowing the mass of ghostly neutrinos — possibly paving the way to new physics: https://www.livescience.com/physics-mathematics/scientists-are-one-step-closer-to-knowing-the-mass-of-ghostly-neutrinos-possibly-paving-the-way-to-new-physics
